Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import mne
import pickle

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
ch_names = ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8']

ch_types = ['eeg'] * 8 + ['misc'] * 2

# Create the info structure needed by MNE
info = mne.create_info(ch_names, ch_types=ch_types, sfreq=250) # 250 Hz

# Data path
data_path = '../data/recordings/combined_recording_1701567996.csv'

user_frame = pd.read_csv(data_path, sep=',')

print("RAW DATA: ")
print(user_frame.shape)
data = user_frame.transpose().to_numpy()

raw = mne.io.RawArray(data, info)

print("RAW SHAPE: ")
print(raw)

raw.set_montage('standard_1020')

In [ ]:
# converts the data from microvolts to volts, required by MNE
raw.apply_function(lambda x: x * 1e-6);

# bandpass filter
raw.filter(2, 30, method='fft')

# isolate the MW events
# events = mne.find_events(raw, stim_channel='current_image_type')

# # isolate button press
# i = 0
# for e in events:
#     events = np.insert(events, i, [e[0]-15000,2,0], axis=0)
#     i+=2

# for e in events:
#     if e[2] == 1:
#         e[1] = 1
#         e[2] = 0

events = mne.find_events(raw, stim_channel='current_image_type')

# # isolate on task events
# i = 0
# for e in events:
#     events = np.insert(events, i, [e[0]-15000,0,2], axis=0)
#     i+=2

event_id = {'real': 1, 'ai': 2}

# make epochs
epochs = mne.Epochs(raw, events, event_id, 0, 10, proj=False, baseline=None, detrend=1, preload=True, event_repeated='drop',
                    reject = dict(eeg=1e-3)
                    )

labels = []
psds = []

labels.append(epochs.events[:, -1]) # append labels for each bin

psd = epochs.compute_psd(method='multitaper', tmin=0, tmax=10, fmin=2, fmax=30, picks='eeg')

# Convert to dB, reshape
psd = 10 * np.log10(psd)
reshaped_psd = psd.reshape(len(psd), len(psd[0]) * len(psd[0][0])) # convert the data into a two dimensional array
psds.append(reshaped_psd)

new_labels = []

#clf = LinearDiscriminantAnalysis()
clf = RidgeClassifier()
sam = np.asarray(psds)
lab = np.asarray(labels)
preds = np.empty(len(labels))

#sam_sparse = coo_matrix(sam)
#sam, sam_sparse, lab = shuffle(sam, sam_sparse, lab, random_state=42)

sam_train, sam_test, lab_train, lab_test = train_test_split(sam[0], lab[0], test_size=0.25, random_state=42)
clf.fit(sam_train, lab_train)
savefile = '../data/classifier_save.sav'
pickle.dump(clf, open(savefile, 'wb'))
preds = clf.predict(sam_test)

report = classification_report(lab_test, preds)
acc = accuracy_score(lab_test, preds)

print("The accuracy for this participant: " + str(acc * 100) + "% \n")